In [1]:
import os
import glob
import gzip
import folium
from fitparse import FitFile

import numpy as np
import pandas as pd

In [3]:
name = 'KM_Strava'
root = f'../data/{name}/activities/'
fit_files = [x for x in os.listdir(root) if x.endswith(".fit.gz")]

In [4]:
def semicir_to_degs(semicirc):
    return semicirc * (180 / 2**31)

def read_fitfile(filename):
    try:
        fitfile = FitFile(gzip.open(filename))
        df = pd.DataFrame([{d['name']: d['value'] for d in r.as_dict()['fields']} 
                                   for r in fitfile.get_messages('record')])
        df['position_lat'] = df['position_lat'].map(semicir_to_degs)
        df['position_long'] = df['position_long'].map(semicir_to_degs)
        df.set_index("timestamp", inplace=True)
        return df
    except Exception as e:
        print(e)
    

In [5]:
central_lat = 39.5744
central_long = -106.0975
m = folium.Map(location=[central_lat,central_long],tiles="Stamen Terrain",zoom_start=12) 

In [7]:
def plot_routes(activity_lst, folder_name, central_lat, central_long):
    m = folium.Map(location=[central_lat,central_long],tiles="Stamen Terrain",zoom_start=12)
    for activity in activity_lst:
        locations = read_fitfile(f'../data/{folder_name}/activities/{activity}')[['position_lat', 'position_long']].dropna()
        points = locations.values.tolist()
        folium.PolyLine(points, color="red", weight=2.5, opacity=0.5).add_to(m)
    return m

plot_routes(fit_files[:10], name, central_lat, central_long)
